In [2]:
import os
import pypdf
import requests
from bs4 import BeautifulSoup
import re

def download_file(url):
    local_filename = url.split('/')[-1]
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                f.write(chunk)
    return local_filename



In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Create a new instance of the Chrome driver
driver = webdriver.Chrome()

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [ ]:
url = "https://www.mc.mil/Cases/MC-Cases"
# the difficulty is that the urls of the pdfs I want are being dynamically exposed via javascript.
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

print(soup)

In [ ]:
testpdf = pypdf.PdfReader
page = testpdf.pages[0]
print(page.extract_text())